In [1]:
import tensorflow as tf

In [3]:
print(tf.add(1,4))

tf.Tensor(5, shape=(), dtype=int32)


In [ ]:
tf.random.learned_unigram_candidate_sampler()

In [ ]:
tf.nn.learned_unigram_candidate_sampler(true_classes=,
    num_true=,
    num_sampled=1,
    unique=False,
    range_max=)